In [ ]:
import geopyspark as gps
import numpy as np
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from geonotebook.wrappers import TMSRasterData

conf = gps.geopyspark_conf(appName="gps-osm-ingest", master='local[8]')
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
conf.set('spark.ui.enabled', True)
conf.set('spark.default.parallelism', 8)
conf.set('spark.master.memory', '9500M')

sc = SparkContext(conf=conf)
pysc = gps.get_spark_context()
session = SparkSession.builder.config(conf=pysc.getConf()).enableHiveSupport().getOrCreate()

In [ ]:
file_uri = "file:/Users/eugene/Downloads/isle-of-man.orc"
osm_dataframe = session.read.orc(file_uri)

In [ ]:
osm = gps.vector_pipe.osm_reader.from_dataframe(osm_dataframe)
lines = osm.get_line_features_rdd()

In [ ]:
highways = lines.filter(
    lambda feature: 'highway' in feature.properties.tags)

In [ ]:
highways.take(1)[0].properties.tags

In [ ]:
# Encode highway speeds as feature properties for rasterization
highway_features = highways.map(
    lambda feature:    
        gps.Feature(feature.geometry, get_highway_cellvalue(feature)))

In [ ]:
highway_features.take(1)

In [ ]:
highway_raster = gps.geotrellis.rasterize_features(
    features = highway_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    num_partitions = 16
).persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
# insert block that renders the lines

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(0, 3, 1), 
    color_list = gps.get_colors_from_matplotlib('magma'))

osm_wm = highway_raster.tile_to_layout(
    gps.GlobalLayout(), target_crs=3857)
layer = gps.TMS.build(osm_wm.pyramid(), color_map)

M.add_layer(TMSRasterData(layer), name="OSM")

In [ ]:
isle_extent = gps.Extent(xmin=-4.921879863281248, ymin=53.964837753906245, 
                         xmax=-4.218754882812476, ymax=54.4921814453125)
#isle_extent = highway_raster.layer_metadata.extent

# Read SRTM for the same extent as rasterized OSM features
srtm = gps.query(
    uri="s3://geotrellis-test/dg-srtm",
    layer_name="srtm-wsg84-gps", 
    layer_zoom = 0,
    query_geom = isle_extent,
    query_proj = 4326,
    num_partitions = 16
)

# Tile SRTM layer to same layout as rasterized OSM features
tiled_srtm = srtm.tile_to_layout(highway_raster.layer_metadata)

In [ ]:
zfactor = gps.geotrellis.zfactor_lat_lng_calculator('Meters')
tobler_raster = tiled_srtm.slope(zfactor).tobler()
    
friction = gps.local_max(highway_raster, tobler_raster)

In [ ]:
reprojected = friction.tile_to_layout(
    target_crs = 3857,
    layout = gps.GlobalLayout(tile_size=256),
    resample_method = gps.ResampleMethod.MAX
).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
pyramid = reprojected.pyramid().persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(0, 7.5, 0.2), 
    color_list = gps.get_colors_from_matplotlib('magma'))

layer = gps.TMS.build(pyramid, color_map)

M.add_layer(TMSRasterData(layer), name="ToblerOSM")

In [ ]:
pyramid.get_histogram().min_max()

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
reprojected.unpersist()

In [ ]:
# Write pyramid to the catalog
for layer in pyrmaid.levels.values():
    gps.write("s3://geotrellis-test/dg-osm-test", "gps-osm-ingest", layer)

In [ ]:
default_speeds = {
'motorway':100,
'trunk':70,
'primary':65,
'secondary':60,
'tertiary':50,
'unclassified':30,
'residential':30,
'service':20,
'motorway_link':70,
'trunk_link':65,
'primary_link':60,
'secondary_link':50,
'tertiary_link':40,
'living_street':10,
'pedestrian':6,
'track':20,
'road':35}

country_speeds = {
'at:urban':50,
'at:rural':100,
'at:trunk':100,
'at:motorway':130,
'ch:urban':50,
'ch:rural':80,
'ch:trunk':100,
'ch:motorway':120,
'cz:urban':50,
'cz:rural':90,
'cz:trunk':130,
'cz:motorway':130,
'dk:urban':50,
'dk:rural':80,
'dk:motorway':130,
'de:living_street':7,
'de:urban':50,
'de:walk':7,
'de:rural':100,
'fi:urban':50,
'fi:rural':80,
'fi:trunk':100,
'fi:motorway':120,
'fr:urban':50,
'fr:rural':90,
'fr:trunk':110,
'fr:motorway':130,
'hu:urban':50,
'hu:rural':90,
'hu:trunk':110,
'hu:motorway':130,
'it:urban':50,
'it:rural':90,
'it:trunk':110,
'it:motorway':130,
'jp:national':60,
'jp:motorway':100,
'ro:urban':50,
'ro:rural':90,
'ro:trunk':100,
'ro:motorway':130,
'ru:living_street':20,
'ru:rural':90,
'ru:urban':60,
'ru:motorway':110,
'sk:urban':50,
'sk:rural':90,
'sk:trunk':130,
'sk:motorway':130,
'si:urban':50,
'si:rural':90,
'si:trunk':110,
'si:motorway':130,
'se:urban':50,
'se:rural':70,
'se:trunk':90,
'se:motorway':110,
'gb:nsl_single':96.54,
'gb:nsl_dual':112.63,
'gb:motorway':112.63,
'ua:urban':60,
'ua:rural':90,
'ua:trunk':110,
'ua:motorway':130,
'living_street':6}

words = ['maxspeed', 'ambiguous', 'signals', 
         'none', 'walk', 'variable', 
         'national', 'fixme', 'unposted', 'implicit']

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def default_speed(highway):
    if not highway in default_speeds:
        return default_speeds['road']
    else:
        return default_speeds[highway]

def get_maxspeed(speed, units, highway):
    speeds = speed.split(';|,-')
    maxspeed = 0
    for sp in speeds:
        sp = sp.replace(units, '')
        if (is_number(sp)):
            if units == 'mph':
                sp = 1.609 * float(sp) 
            elif units == 'knots':
                sp = 1.852 * float(knots)
            else:
                sp = float(sp)
                
            if sp > maxspeed:
                maxspeed = sp
    if maxspeed > 0:
        speed = maxspeed
    else:
        speed = default_speed(highway)

    return speed

def get_highway_cellvalue(osm_feature):   
    highway = osm_feature.properties.tags['highway']
    speed = osm_feature.properties.tags.get('maxspeed', '')
                                
    speed = speed.lower().strip()
        
    # if we don't have a speed, give it a default
    if len(speed) == 0:
        speed = default_speed(highway)
    elif not is_number(speed):
        if 'kph' in speed:
            speed = get_maxspeed(speed, 'kph', highway)
        elif 'km/h' in speed:
            speed = get_maxspeed(speed, 'km/h', highway)
        elif 'kmh' in speed:
            speed = get_maxspeed(speed, 'kmh', highway)
        elif 'mph' in speed:
            speed = get_maxspeed(speed, 'mph', highway)
        elif 'knots' in speed:
            speed = get_maxspeed(speed, 'knots', highway)
        elif speed in country_speeds:
            speed = country_speeds[speed]
        elif speed in words:
            speed = default_speed(highway)
        else:
            speed = get_maxspeed(speed, '', highway)            
    if float(speed) <= 0.0:
        speed = default_speed(highway)

    speed = float(speed)
    return gps.CellValue(speed, speed)